In [29]:
import numpy as np

import pandas as pd
pd.options.display.max_rows = 4000

In [30]:
data=pd.read_csv("ADMISSIONS.csv")
icd=pd.read_csv("D_ICD_DIAGNOSES.csv")
diag=pd.read_csv("DIAGNOSES_ICD.csv")

In [31]:
ldata=pd.merge(icd, diag, on="ICD9_CODE")

ldata= ldata.drop(columns=['ROW_ID_x', 'ICD9_CODE', 'LONG_TITLE', 'ROW_ID_y',
                           'SUBJECT_ID', 'SEQ_NUM'])

ldata=pd.merge(ldata, data, on="HADM_ID")
ldata.head()

,SHORT_TITLE,HADM_ID,ROW_ID,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,Pulmon TB NEC-oth test,188604,54015,84318,2195-02-20 19:17:00,2195-04-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,SPAN,CATHOLIC,MARRIED,HISPANIC OR LATINO,2195-02-20 13:54:00,2195-02-20 20:46:00,NAUSEA W\VOMITING,0,1
1,Oth gram negatv bacteria,188604,54015,84318,2195-02-20 19:17:00,2195-04-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,SPAN,CATHOLIC,MARRIED,HISPANIC OR LATINO,2195-02-20 13:54:00,2195-02-20 20:46:00,NAUSEA W\VOMITING,0,1
2,Cutaneous candidiasis,188604,54015,84318,2195-02-20 19:17:00,2195-04-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,SPAN,CATHOLIC,MARRIED,HISPANIC OR LATINO,2195-02-20 13:54:00,2195-02-20 20:46:00,NAUSEA W\VOMITING,0,1
3,Mycoses NEC & NOS,188604,54015,84318,2195-02-20 19:17:00,2195-04-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,SPAN,CATHOLIC,MARRIED,HISPANIC OR LATINO,2195-02-20 13:54:00,2195-02-20 20:46:00,NAUSEA W\VOMITING,0,1
4,Ac myl leuk wo achv rmsn,188604,54015,84318,2195-02-20 19:17:00,2195-04-03 14:00:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Medicare,SPAN,CATHOLIC,MARRIED,HISPANIC OR LATINO,2195-02-20 13:54:00,2195-02-20 20:46:00,NAUSEA W\VOMITING,0,1


Get only data on newborn children admitted to neonatal ICU:

In [32]:
ndata=ldata[ldata['ADMISSION_TYPE'] == 'NEWBORN']
ndata.head()

,SHORT_TITLE,HADM_ID,ROW_ID,SUBJECT_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
25808,Viral enteritis NOS,138701,22436,18374,2142-03-13 12:57:00,2142-04-14 13:40:00,NaN,NEWBORN,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Private,NaN,UNOBTAINABLE,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,PREMATURITY,0,1
25809,Perinatal chr resp dis,138701,22436,18374,2142-03-13 12:57:00,2142-04-14 13:40:00,NaN,NEWBORN,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Private,NaN,UNOBTAINABLE,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,PREMATURITY,0,1
25810,Failure to thrive-child,138701,22436,18374,2142-03-13 12:57:00,2142-04-14 13:40:00,NaN,NEWBORN,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Private,NaN,UNOBTAINABLE,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,PREMATURITY,0,1
25811,Hx-congenital malfor NEC,138701,22436,18374,2142-03-13 12:57:00,2142-04-14 13:40:00,NaN,NEWBORN,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Private,NaN,UNOBTAINABLE,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,PREMATURITY,0,1
28634,Early congen syph NOS,126409,24799,20311,2190-06-17 14:50:00,2190-06-24 12:00:00,NaN,NEWBORN,PHYS REFERRAL/NORMAL DELI,HOME,Medicaid,NaN,UNOBTAINABLE,NaN,ASIAN,NaN,NaN,NEWBORN,0,1


In [33]:
len(ndata['HADM_ID'].unique()) # number of newborns

7861

In [34]:
#add length of stay to dataframe:

los=pd.read_csv('ICUSTAYS.csv') 
ndata=pd.merge(ndata, los, on="HADM_ID")

Get only infants whose stay is longer than 2 days. This is to remove most infants who are not admitted to NICU with a serious health insult, such as those going there for a routine circumsision:

In [35]:
ndata=ndata[ndata['LOS'] >= 48]

In [36]:
len(ndata['HADM_ID'].unique()) # number after removing those with short stays.

451

Get counts of common ailments:

In [37]:
diagg=pd.DataFrame(ndata['SHORT_TITLE'].value_counts())

In [38]:
print(diagg)

                          SHORT_TITLE
Neonat jaund preterm del          408
Respiratory distress syn          381
Primary apnea of newborn          256
Anemia of prematurity             227
Need prphyl vc vrl hepat          224
NB obsrv suspct infect            202
Patent ductus arteriosus          178
Neonatal bradycardia              170
Single lb in-hosp w cs            167
Perinatal chr resp dis            139
Twin-mate lb-in hos w cs          135
Retrolental fibroplasia           131
27-28 comp wks gestation          126
NB septicemia [sepsis]            107
Perinatal condition NEC           106
Preterm NEC 1000-1249g            105
29-30 comp wks gestation          105
Extreme immatur 750-999g           97
Single lb in-hosp w/o cs           80
Preterm NEC 1250-1499g             79
Routine circumcision               78
Extreme immatur 500-749g           74
25-26 comp wks gestation           73
Late metab acidosis NB             55
NB intraven hem,grade i            54
Secundum atr

Based on the breakdown of aliments affecting newborns, difficulties with the respiratory system and circulatory system are very prevalent among preterm and non-preterm infants as reasons for admission to the ICU. As such, we decided to focus on the effect that these conditions, as well as the degree to which an infant was premature, had on the length of stay in the ICU and their discharge location. As the number of diagnoses is large, we categorized insults to these systems into 5 categories of disease. Only insults that appeared in at least 10 diagnoses overall were included in categorization. These categories are:

- **cardiac hemorrage (1/0)**: included were conditions such as intraventricular hemorrages, pulmonary hemorrages.
- **circulatory system malformation (1/0)**: included were conditions such as patent ductus arteriosus, ventricular septal defect.
- **heart rate/circulation issues (1/0)**: Included were conditons such as neonatal bradycardia, hypotension.
- **preterm** (0 = not preterm, 1 = preterm, 2 = extreme preterm (defined in database as < 1kg)
- **respiratory distress (1/0)**: included were conditiions such as atelectasis, apnea, respiratory distress syndrome.
- **sepsis (1/0)**
- **jaundice (1/0)**: Though the diagnosis is listed as 'preterm jaundice' this label; is applied to many non-preterm infants, suggesting it is used as a generic jaundice diagnosis for infants. As such, we have considered it as such. 

Patients can be diagnosed with multiple of the above conditions simultaneously.

In [39]:
diagg=diagg.reset_index()

In [40]:
#this is the full list of conditions which had at least 10 occurrences, and were pertinent to the above categories:

dg=['Respiratory distress syn', 'Primary apnea of newborn','Patent ductus arteriosus', 'Neonatal bradycardia', 
    'Anemia of prematurity','Perinatal chr resp dis','Preterm NEC 1000-1249g', 'Extreme immatur 750-999g','Preterm NEC 1250-1499g', 
    'Extreme immatur 500-749g','NB intraven hem,grade i','Secundum atrial sept def', 'Other apnea of newborn',
   'Preterm NEC 1500-1749g','NB interstit emphysema','Cong pulmon valve stenos','Ventricular sept defect',
   'Resp prob after brth NEC', 'Hypotension NOS', 'NB intraven hem,grade ii','Preterm NEC 750-999g', 'NB intraven hem NOS', 
    'Ab ftl hrt rt/rhy NOS','NB transitory tachypnea', 'Preterm NEC 500-749g', 'NB atelectasis NEC/NOS', 
    'NB intravn hem,grade iii','NB pulmonary hemorrhage', 'Neonat jaund preterm del', 'NB septicemia [sepsis]']

diagCat=ndata.query('SHORT_TITLE in @dg')

In [41]:
#this is how each of the above was categorized:

cat=['respiratory','respiratory','circulatory system malformation',
                     'heart rate/circulation issues','heart rate/circulation issues','respiratory', 'preterm',
                     'extreme preterm','preterm','extreme preterm',
                     'cardiac hemorrage', 'circulatory system malformation', 
                     'respiratory', 'preterm','respiratory', 'circulatory system malformation',
                    'circulatory system malformation', 'respiratory',
                     'heart rate/circulation issues', 'cardiac hemorrage',
                     'preterm', 'cardiac hemorrage','heart rate/circulation issues',
                     'respiratory', 'preterm','respiratory','cardiac hemorrage','cardiac hemorrage', 'jaundice', 'sepsis']
diagCat=diagCat.replace(dg, cat)

In [42]:
#breakdown of ailment inclusion and categorization:

for i in range(len(dg)):
    print(dg[i], "was categorized as", cat[i])

Respiratory distress syn was categorized as respiratory
Primary apnea of newborn was categorized as respiratory
Patent ductus arteriosus was categorized as circulatory system malformation
Neonatal bradycardia was categorized as heart rate/circulation issues
Anemia of prematurity was categorized as heart rate/circulation issues
Perinatal chr resp dis was categorized as respiratory
Preterm NEC 1000-1249g was categorized as preterm
Extreme immatur 750-999g was categorized as extreme preterm
Preterm NEC 1250-1499g was categorized as preterm
Extreme immatur 500-749g was categorized as extreme preterm
NB intraven hem,grade i was categorized as cardiac hemorrage
Secundum atrial sept def was categorized as circulatory system malformation
Other apnea of newborn was categorized as respiratory
Preterm NEC 1500-1749g was categorized as preterm
NB interstit emphysema was categorized as respiratory
Cong pulmon valve stenos was categorized as circulatory system malformation
Ventricular sept defect wa

In [43]:
drop=['ROW_ID_y', 'ROW_ID_x','ADMISSION_TYPE', 'DBSOURCE', 'INSURANCE','LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY','HAS_CHARTEVENTS_DATA','SUBJECT_ID_y', 'EDREGTIME', 'EDOUTTIME',
     'DIAGNOSIS']
diagCat=diagCat.drop(drop, axis=1)

Pivot table to get categories in presence/absence format, and get 1 row per patient:

In [44]:
ccc=diagCat.iloc[:,:2]
ccc=ccc.drop_duplicates()
ccc['val']=1
ccc=ccc.pivot(index='HADM_ID', columns='SHORT_TITLE')['val']
ccc=ccc.fillna(0)
ccc=ccc.reset_index()

In [45]:
ccc.head()
#we have separate columns for preterm and extreme preterm here, but we collapse this into 1 further down

SHORT_TITLE,HADM_ID,cardiac hemorrage,circulatory system malformation,extreme preterm,heart rate/circulation issues,jaundice,preterm,respiratory,sepsis
0,100135,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
1,100419,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2,100545,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,100591,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
4,100665,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [46]:
g=ndata[['SUBJECT_ID_x','HADM_ID','DISCHARGE_LOCATION', 'LOS']]

In [47]:
ccc=pd.merge(ccc, g, on="HADM_ID")
ccc=ccc.drop_duplicates()

In [48]:
#add infant sex to dataframe
ind=pd.read_csv('PATIENTS.csv')

In [49]:
ccc = ccc.rename(columns={'SUBJECT_ID_x': 'SUBJECT_ID'})

In [50]:
ccc=pd.merge(ccc, ind, on='SUBJECT_ID')

In [51]:
drop=['SUBJECT_ID', 'ROW_ID', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 'EXPIRE_FLAG']
ccc=ccc.drop(drop, axis=1)

categorize preterms into 3 categories:

In [52]:
mask = (ccc['extreme preterm'] == 1)
ccc['preterm'][mask] = 2

C:\Users\lukec\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [53]:
ccc=ccc.drop('extreme preterm', axis=1)

Add column for censorship. Only 1 death, so only 1 patient in censored in terms of their leaving the ICU:

In [54]:
ccc['delta']=1
mask = (ccc['DISCHARGE_LOCATION'] == 'DEAD/EXPIRED')
ccc['delta'][mask] = 0

C:\Users\lukec\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
#Final dataframe:
ccc.head()

,HADM_ID,cardiac hemorrage,circulatory system malformation,heart rate/circulation issues,jaundice,preterm,respiratory,sepsis,DISCHARGE_LOCATION,LOS,GENDER,delta
0,100135,0.0,0.0,1.0,1.0,1.0,1.0,0.0,HOME HEALTH CARE,96.4259,F,1
1,100419,0.0,1.0,1.0,1.0,1.0,1.0,1.0,HOME HEALTH CARE,78.2365,M,1
2,100545,0.0,0.0,0.0,1.0,2.0,1.0,0.0,HOME HEALTH CARE,84.9136,M,1
3,100591,0.0,1.0,1.0,1.0,1.0,1.0,0.0,SHORT TERM HOSPITAL,55.3711,M,1
4,100665,0.0,0.0,1.0,0.0,1.0,1.0,0.0,HOME HEALTH CARE,49.3126,M,1


In [56]:
#export_csv = ccc.to_csv ('datadfpretermNew.csv', index = None, 
 #                              header=True)